In [113]:
import numpy as np
from random import randint

In [114]:
tam_obstaculo = 10
random = 2
coordenada_inicial = [0, 0]
coordenada_final = [20, 20]

# 1º Etapa - Criar os obstaculos e os pontos iniciais e finais

Essa etapa consiste em criar um mapa de proporção MxN, capaz de alocar todos os obstaculos e os pontos iniciais e finais;

Observações:

- Todos os obstaculos devem ser gerados automaticamente
- Todos os obstaculos devem estar dentro do espaço do mapa (Caso aconteça, sortear uma nova posição)
- Nenhum obstaculo deve sobrepor o outro (Caso aconteça, sortear uma nova posição)

- para coordenadas, seguir a seguinte ordem para os valores em um array

0-----1 <br>
2-----3


In [115]:
'''
Verificar se a coordenada x e y de um dado ponto está dentro do rentagulo válido do mapa
retorna True caso seja no intervalo e False caso a coordenada seja inválida
'''
def coordenada_valida(ponto, x_min_mapa, y_min_mapa, x_max_mapa, y_max_mapa):
    '''
    ponto: [x,y]
    x_min: Coordenada x minima do mapa
    y_min: Coordenada y minima do mapa
    x_max: Coordenada x máxima do mapa
    y_max: Coordenada y máxima do mapa
    '''
    return x_min_mapa <= ponto[0] <= x_max_mapa and y_min_mapa <= ponto[1] <= y_max_mapa   

In [ ]:
'''
Verifica se o obstaculo criado sobrepoe algum dos obstaculos criados anteriormentes
Deve retornar True ou False

Verificar se o x ou y de cada ponto do novo obstaculo está dentro do espaço compreendido no obstaculo já criado

Posso usar o obstaculo já criado como o “mapa” e analisar se pelo menos um ponto do novo obstaculo está 
compreendido dentro do obstaculo analisado (fazer cada verificação com o coordenada_valida)


'''
def obstaculo_sobreposto():
    '''
    novo_obstaculo: lista com os pontos do novo obstaculo -> [[x,y], [x,y], [x,y], [x,y]]
    obstaculos: lista com todos os obstaculos já criados -> [[[x,y], [x,y], [x,y], [x,y]] , [[x,y], [x,y], [x,y], [x,y]]]
    
    
    retornar True se os obstaculos se sobrepoe. False caso não se sobrepoe.
    '''

In [126]:
'''


'''
def criar_mapa(tam_obstaculo, qtd_obstaculos, coordenada_inicial, coordenada_final):
    '''
    tam_obstaculo: valor que corresponde a proporção do obstaculo. ex: 10 (obstaculo 10x10)
    qtd_obstaculos: quantidade de obstaculos a serem gerados
    coordenada_inicial: coordenada x,y do ponto de partida
    coordenada_final: coordenada x,y do ponto final
    '''
    try:
        
        max_mapa = qtd_obstaculos * tam_obstaculo
        
        mapa = np.array([[0,0], [0, max_mapa], [max_mapa, 0], [max_mapa, max_mapa]])
        
        # verificar se as coordenadas de inicio estão dentro do mapa
        if not coordenada_valida(coordenada_inicial, mapa[0][0], mapa[0][1], mapa[3][0], mapa[3][1]):
            raise ValueError('Coordenada incial inválida')
            
        # verificar se as coordenadas final estão dentro do mapa
        if not coordenada_valida(coordenada_final, mapa[0][0], mapa[0][1], mapa[3][0], mapa[3][1]):
            raise ValueError('Coordenada Final inválida')
            
        '''
        Criar obstaculos 
        
        - Sortear um ponto x,y com tamanho maximo e minimo compreendido no tamanho do mapa
        - verificar se esse ponto com o tamanho do obstaculo nao ultrapassa o espaço do mapa
        '''    
        
        x = 0
        obstaculos = []
        while x < qtd_obstaculos:
            x_min_obstaculo = randint(mapa[0][0], mapa[3][1])
            y_min_obstaculo = randint(mapa[0][0], mapa[3][1])
            
            ponto1 = [x_min_obstaculo, y_min_obstaculo]
            ponto2 = [x_min_obstaculo, y_min_obstaculo + tam_obstaculo]
            ponto3 = [x_min_obstaculo + tam_obstaculo, y_min_obstaculo]
            ponto4 = [x_min_obstaculo + tam_obstaculo, y_min_obstaculo + tam_obstaculo]
            
            # Verificar se algum ponto do obstaculo ultrapassa o mapa
            # caso ultrapassar, essa repetição será ignorada (continue) e será gerado uma nova posição para o retangulo
            if not coordenada_valida(ponto2, mapa[0][0], mapa[0][1], mapa[3][0], mapa[3][1]):
                continue
                
            if not coordenada_valida(ponto3, mapa[0][0], mapa[0][1], mapa[3][0], mapa[3][1]):
                continue
                
            if not coordenada_valida(ponto4, mapa[0][0], mapa[0][1], mapa[3][0], mapa[3][1]):
                continue
                
            # verificar se algum obstaculo sobrepoe outro já existente
            
            
            # pontos do obstaculo válido, criar lista com todos os pontos
            obstaculos.append([ponto1, ponto2, ponto3, ponto4])
            x += 1
            
        resultados = {}
        resultados['mapa'] = mapa
        resultados['coordenada_inicial'] = coordenada_inicial
        resultados['coordenada_final'] = coordenada_final
        resultados['obstaculos'] = obstaculos
        
        return resultados
    except ValueError as e:
        print(f'Error: {e}')

In [130]:
mapa = criar_mapa(tam_obstaculo, random, coordenada_inicial, coordenada_final)
mapa

{'mapa': array([[ 0,  0],
        [ 0, 20],
        [20,  0],
        [20, 20]]),
 'coordenada_inicial': [0, 0],
 'coordenada_final': [20, 20],
 'obstaculos': [[[10, 2], [10, 12], [20, 2], [20, 12]],
  [[5, 0], [5, 10], [15, 0], [15, 10]]]}